In [1]:
! pip install numpy pandas
! pip install -U gretel-client


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.6/58.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.7/105.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: tabulate
    Found existing installation: tabulate 0.9.0
    Uninstalling tabulate-0.9.0

In [2]:
from gretel_client import configure_session
configure_session(api_key="prompt", validate=True, clear=True)

Gretel API Key: ··········
Using endpoint https://api.gretel.cloud
Logged in as chinie876@gmail.com ✅


In [3]:
import pandas as pd
DATA_PATH = "https://gretel-public-website.s3.us-west-2.amazonaws.com/datasets/uci_diabetes_readmission_data.csv"
df = pd.read_csv(DATA_PATH)

In [4]:
pd.set_option("display.max_columns", 50)
df.head()

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,2278392,8222157,Caucasian,Female,[0-10),6,25,1,1,Pediatrics-Endocrinology,41,0,1,0,0,0,1,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,NO
1,149190,55629189,Caucasian,Female,[10-20),1,1,7,3,?,59,0,18,0,0,0,9,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,>30
2,64410,86047875,AfricanAmerican,Female,[20-30),1,1,7,2,?,11,5,13,2,0,1,6,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
3,500364,82442376,Caucasian,Male,[30-40),1,1,7,2,?,44,1,16,0,0,0,7,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
4,16680,42519267,Caucasian,Male,[40-50),1,1,7,1,?,51,0,8,0,0,0,5,NaN,NaN,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,NO


In [5]:
from gretel_client.projects import create_or_get_unique_project

# set up a project
project = create_or_get_unique_project(name="hospital-readmission-tabular-dp")

# upload data source
data_source_identifier = project.upload_artifact(DATA_PATH)

In [6]:
from gretel_client.projects.models import read_model_config
from pprint import pprint
from gretel_client.helpers import poll

# Create a new model configuration.
config = read_model_config("synthetics/tabular-differential-privacy")
config["models"][0]["tabular_dp"]["params"]["epsilon"] = 0.5
config["name"] = "hospital-readmission-tabular-dp-epsilon-0.5"

# view config
pprint(config)

# create and submit the model for training
model = project.create_model_obj(model_config=config, data_source=data_source_identifier)
model.submit_cloud()
poll(model)

# view the synthetic data generated
synthetic = pd.read_csv(model.get_artifact_link("data_preview"), compression="gzip")
display(synthetic.head())

# get quick information on synthetic data quality
pprint(model.get_report_summary())

{'models': [{'tabular_dp': {'data_source': '__tmp__',
                            'params': {'delta': 'auto',
                                       'epsilon': 0.5,
                                       'infer_domain': True}}}],
 'name': 'hospital-readmission-tabular-dp-epsilon-0.5',
 'schema_version': '1.0'}


INFO: Starting poller


{
    "uid": "66165e453f1dcb7da8a7d223",
    "guid": "model_2eu68gt1WVizyJtgVyNLJAcjeuo",
    "model_name": "hospital-readmission-tabular-dp-epsilon-0.5",
    "model_key": "",
    "runner_mode": "cloud",
    "user_id": "64e3290ebff6212a9d0c236d",
    "user_guid": "user_2UHuMyeIIbRvDa3BKSr5JQivZB2",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "66165d912eeeabfde59522c5",
    "project_guid": "proj_2eu5m1sopByZwROV2JdQdCevgq3",
    "cluster_guid": null,
    "status_history": {
        "created": "2024-04-10T09:39:17.219986Z"
    },
    "last_modified": "2024-04-10T09:39:17.306065Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "provenance": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/tabular_dp@sha256:496c478df72190c7e9cef7812a7717bf27da77915538ff716ae0a34a98dc35e1",
    "container_ima

INFO: Status is created. Model creation has been queued.
INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2024-04-10T09:39:27.042176Z  Analyzing input data and checking for auto-params...
2024-04-10T09:39:27.043218Z  Parameter `delta` was automatically set to 3.1e-08 based on the number of records, n. Note that n was not determined with differential privacy.
2024-04-10T09:39:27.043575Z  Found 1 auto-params that were set based on input data.
{
    "delta": 3.080320762718042e-08
}
2024-04-10T09:39:27.044063Z  Using updated model configuration: 
{
    "schema_version": "1.0",
    "name": "hospital-readmission-tabular-dp-epsilon-0.5",
    "models": [
        {
            "tabular_dp": {
                "data_source": [
                    "gretel_ba71d5444517457d83e1826b8cf3e81e_uci_diabetes_readmission_data.csv"
                ],
                "ref_data": {},
                "pa

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,286159345,88613224,Caucasian,Female,[50-60),1,11,7,12,?,13,1,15,1,0,0,9,NaN,NaN,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
1,295543217,115360873,Caucasian,Female,[70-80),1,1,7,1,InternalMedicine,69,0,16,0,0,0,9,>300,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,Yes,>30
2,402591173,157284741,Caucasian,Male,[60-70),1,1,7,6,?,67,0,14,0,0,1,9,NaN,NaN,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,Steady,No,No,No,No,No,Ch,Yes,<30
3,106520472,77766531,Caucasian,Male,[50-60),3,1,1,4,Orthopedics-Reconstructive,42,0,9,0,3,0,5,NaN,NaN,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,NO
4,165258146,58606015,Caucasian,Female,[80-90),2,3,5,6,Surgery-General,30,0,5,0,0,2,9,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,>30


{'summary': [{'field': 'synthetic_data_quality_score', 'value': 89},
             {'field': 'field_correlation_stability', 'value': 92},
             {'field': 'principal_component_stability', 'value': 79},
             {'field': 'field_distribution_stability', 'value': 97},
             {'field': 'privacy_protection_level', 'value': 6}]}


In [8]:
import pandas as pd

from gretel_client.config import RunnerMode
from gretel_client.evaluation.quality_report import QualityReport
from gretel_client import configure_session
from gretel_client.projects import create_or_get_unique_project

In [9]:

report = QualityReport(data_source=synthetic, ref_data=df)
report.run()

import IPython

IPython.display.HTML(report.as_html, metadata=dict(isolated=True))

How to interpret your SQS,Excellent,Good,Moderate,Poor,Very Poor
Suitable for machine learning or statistical analysis,,,,,
Suitable for balancing or augmenting machine learning data sources,,,,,
Suitable for pre-production testing environments,,,,,
Suitable for demo environments or mock data,,,,,
Improve your model using our tips and advice,,,,,
Significant tuning required to improve model,,,,,
Data Sharing Use Case,Excellent,Very Good,Good,Normal,Poor
"Internally, within the same team",,,,,
"Internally, across different teams",,,,,
"Externally, with trusted partners",,,,,


In [11]:
# Create a new model configuration.
config2 = read_model_config("synthetics/tabular-differential-privacy")
config2["models"][0]["tabular_dp"]["params"]["epsilon"] = 1
config2["name"] = "hospital-readmission-tabular-dp-epsilon-1"

# view config
pprint(config2)

# create and submit the model for training
model2 = project.create_model_obj(model_config=config2, data_source=data_source_identifier)
model2.submit_cloud()
poll(model2)

# view the synthetic data generated
synthetic2 = pd.read_csv(model2.get_artifact_link("data_preview"), compression="gzip")
display(synthetic2.head())

# get quick information on synthetic data quality
pprint(model2.get_report_summary())



{'models': [{'tabular_dp': {'data_source': '__tmp__',
                            'params': {'delta': 'auto',
                                       'epsilon': 1,
                                       'infer_domain': True}}}],
 'name': 'hospital-readmission-tabular-dp-epsilon-1',
 'schema_version': '1.0'}


INFO: Starting poller


{
    "uid": "6616651a3f1dcb7da8a7d228",
    "guid": "model_2eu9gP2h9asVT8P41BioPsYqs7T",
    "model_name": "hospital-readmission-tabular-dp-epsilon-1",
    "model_key": "",
    "runner_mode": "cloud",
    "user_id": "64e3290ebff6212a9d0c236d",
    "user_guid": "user_2UHuMyeIIbRvDa3BKSr5JQivZB2",
    "billing_domain": null,
    "billing_domain_guid": null,
    "project_id": "66165d912eeeabfde59522c5",
    "project_guid": "proj_2eu5m1sopByZwROV2JdQdCevgq3",
    "cluster_guid": null,
    "status_history": {
        "created": "2024-04-10T10:08:26.615353Z"
    },
    "last_modified": "2024-04-10T10:08:26.797245Z",
    "status": "created",
    "last_active_hb": null,
    "duration_minutes": null,
    "error_msg": null,
    "error_id": null,
    "traceback": null,
    "annotations": null,
    "provenance": null,
    "container_image": "074762682575.dkr.ecr.us-west-2.amazonaws.com/models/tabular_dp@sha256:496c478df72190c7e9cef7812a7717bf27da77915538ff716ae0a34a98dc35e1",
    "container_image

INFO: Status is pending. A Gretel Cloud worker is being allocated to begin model creation.
INFO: Status is active. A worker has started creating your model!
2024-04-10T10:08:36.133495Z  Analyzing input data and checking for auto-params...
2024-04-10T10:08:36.134632Z  Parameter `delta` was automatically set to 3.1e-08 based on the number of records, n. Note that n was not determined with differential privacy.
2024-04-10T10:08:36.134976Z  Found 1 auto-params that were set based on input data.
{
    "delta": 3.080320762718042e-08
}
2024-04-10T10:08:36.135475Z  Using updated model configuration: 
{
    "schema_version": "1.0",
    "name": "hospital-readmission-tabular-dp-epsilon-1",
    "models": [
        {
            "tabular_dp": {
                "data_source": [
                    "gretel_ba71d5444517457d83e1826b8cf3e81e_uci_diabetes_readmission_data.csv"
                ],
                "ref_data": {},
                "params": {
                    "epsilon": 1.0,
              

,encounter_id,patient_nbr,race,gender,age,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,medical_specialty,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,max_glu_serum,A1Cresult,metformin,repaglinide,nateglinide,chlorpropamide,glimepiride,acetohexamide,glipizide,glyburide,tolbutamide,pioglitazone,rosiglitazone,acarbose,miglitol,troglitazone,tolazamide,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmitted
0,92106001,22943723,Caucasian,Female,[60-70),1,3,7,4,Family/GeneralPractice,44,0,7,0,0,0,9,NaN,NaN,Steady,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Yes,NO
1,128931750,39761823,Caucasian,Male,[90-100),1,1,7,4,?,31,0,15,0,0,2,5,NaN,>8,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,Up,No,No,No,No,No,Ch,Yes,NO
2,383427062,56500488,Caucasian,Female,[80-90),1,22,7,11,?,55,2,14,0,0,0,9,NaN,NaN,No,No,No,No,No,No,No,Steady,No,No,No,No,No,No,No,Down,No,No,No,No,No,Ch,Yes,<30
3,116415114,94646084,Caucasian,Female,[70-80),1,6,7,14,InternalMedicine,41,1,26,1,0,1,6,NaN,NaN,No,No,No,No,No,No,Steady,Steady,No,Steady,No,No,No,No,No,No,No,No,No,No,No,Ch,Yes,NO
4,358828219,37303273,Caucasian,Male,[60-70),1,1,7,1,Pulmonology,1,2,7,0,0,0,9,Norm,NaN,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,No,>30


{'summary': [{'field': 'synthetic_data_quality_score', 'value': 92},
             {'field': 'field_correlation_stability', 'value': 92},
             {'field': 'principal_component_stability', 'value': 85},
             {'field': 'field_distribution_stability', 'value': 99},
             {'field': 'privacy_protection_level', 'value': 6}]}
